In [1]:
import time
import ast
from llama_cpp import Llama
import re
import pandas as pd

from IPython.display import display, HTML

In [2]:
llm = Llama(
  model_path="./llm_models/Phi-3-mini-4k-instruct-q4.gguf",
  n_ctx=4096,  # The max sequence length to use
  n_threads=8,
  n_gpu_layers=-1
)

llama_model_loader: loaded meta data with 25 key-value pairs and 291 tensors from ./llm_models/Phi-3-mini-4k-instruct-q4.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 32064
llama_model_loader: - kv   3:                       llama.context_length u32              = 4096
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 3072
llama_model_loader: - kv   5:                          llama.block_count u32              = 32
llama_model_loader: - kv   6:                  llama.feed_forward_length u32              = 8192
llama_model_loader: - kv   7:                 llama.rope.dimension

In [3]:
def getPrompt(taskQuestion: str) -> str:
    return '''
        <|user|>\nYour job is to extract actions and objects from the given task and invoke the allowed functions sequentially. Answer only with action.ignore() if no appropriate function available or you don't know what to do.
        Do not greet, comment, explain, or elaborate your thought processes. Split the task in as many as sub-actions possible. Use semicolon to seperate the function calls. python required.
        Always recognise task language and translate the task and the object name to English. Always check if the function is allowed.
        Allowed functions are: action.pick(object_name), action.pick_place(object_name, target_location), action.ignore()
        \n<|user|>Example 1:
        Task: Pick grapes and keep it on the plate.<|end|>
        \n<|assistant|>action.pick_place("grapes", "plate")<|end|>
        \n<|user|>Example 2:
        Task: Pickup the grapes.<|end|>
        \n<|assistant|>action.pick("grapes")<|end|>
        \nNow translate to English and answer for task: ''' + taskQuestion + '<|end|>\n<|assistant|>'


In [6]:
# taskQuestion = 'throw the coke cane in the garbage bin'
# taskQuestion = 'Put the flower pot on the table'
# taskQuestion = 'Pick up the red ball and place it on the blue box'
# taskQuestion = 'grab the banana and keep it on the plate'
# taskQuestion = 'pickup the banana and dance'
# taskQuestion = 'Hold the soldering iron'
# taskQuestion = 'Keep the apple in the tiffin box'
# taskQuestion = 'गेंद उठाओ और बॉक्स में रखो'
# taskQuestion = 'Tartsa az almát a tiffin dobozban' # hungarian
# taskQuestion = 'Vedd fel a piros la/bdát és tedd a dobozba' # hungarian
# taskQuestion = 'Guarda la manzana en la caja tiffin.' # spanish
# taskQuestion = 'Recoge la pelota y colócala en la caja' # spanish
# taskQuestion = 'Conserva la mela nella scatola del tiffin' # italian

# taskQuestion = 'Pick the banana and then pick the apple'
# taskQuestion = 'Pick the banana and then keep the apple on the box'
taskQuestion = 'place the flower pot in the box, keep the bread on the plate'

start_time = time.perf_counter()
output = llm(
  # f"<|user|>\n{prompt}<|end|>\n<|assistant|>",
  prompt=getPrompt(taskQuestion),
  max_tokens=160,  # Generate up to 256 tokens
  stop=["<|end|>"],
  echo=False,
  temperature=0.4
)
end_time = time.perf_counter()
print("inference time: %.1fms\n" % ((end_time-start_time)*1000,))

print(output['choices'][0]['text'])

Llama.generate: prefix-match hit

llama_print_timings:        load time =    3619.52 ms
llama_print_timings:      sample time =       2.65 ms /    26 runs   (    0.10 ms per token,  9826.15 tokens per second)
llama_print_timings: prompt eval time =     238.35 ms /    17 tokens (   14.02 ms per token,    71.32 tokens per second)
llama_print_timings:        eval time =     627.54 ms /    25 runs   (   25.10 ms per token,    39.84 tokens per second)
llama_print_timings:       total time =     906.14 ms /    42 tokens


inference time: 910.1ms

 action.pick_place("flower pot", "box"); action.pick_place("bread", "plate")


In [140]:
# graspObj = 'guava'        # ANS: vertical cylindrical @center
# graspObj = 'banana'       # ANS: pinch @center
graspObj = 'wine glass'   # ANS: vertical cylindrical @stem base
# graspObj = 'pizza box'    # ANS: horizontal cylindrical @center
# graspObj = 'dice'         # ANS: vertical cylindrical @center / pinch @center
# graspObj = 'pen'          # ANS: pinch @center of barrel
# graspObj = 'knife'        # ANS: pinch @handle

# graspPrompt = f'''
# You are controlling a robotic arm and have to respond how to grip the given object. Tall and narrow objects require vertical cylindrical grasp, small objects require pinch, wide and short objects require horizontal cylindrical grasp.
# You will be given the object name and have to answer the most appropriate grasping method. Do not explain, just give the name of grasp.
# Example:
# Object: Water bottle, Grasp: vertical cylindrical
# Object: Cup, Grasp: pinch
# Object: miniature toy car, Grasp: horizontal cylindrical

# You answer for object: {graspObj}
# '''

graspPrompt = f'''
You are controlling a robotic arm and have to respond how to grip the given object. Tall and narrow objects require vertical cylindrical grasp, small objects require pinch, wide and short objects require horizontal cylindrical grasp.
You will be given the object name and have to answer the most appropriate grasping method along with best position to grab it. Do not provide instruction or explanation or example or comment, just give the name of grasp.

Example:
Object: Water bottle, Answer: vertical cylindrical @middle
Object: Tea cup, Answer: pinch @handle
Object: miniature toy car, Answer: horizontal cylindrical @middle
Object: marble ball, Answer: pinch @any
Object: sword, Answer: pinch @middle
Object: cardboard box, Answer: pinch @edge

You answer for object: {graspObj}
'''


start_time = time.perf_counter()
graspOutput = llm(
  f"<|user|>\n{graspPrompt}<|end|>\n<|assistant|>",
  max_tokens=160,  # Generate up to 256 tokens
  stop=["<|end|>"],
  echo=False,
  temperature=0.4
)
end_time = time.perf_counter()
print("inference time: %.1fms\n" % ((end_time-start_time)*1000,))

print(graspOutput['choices'][0]['text'])

Llama.generate: prefix-match hit

llama_print_timings:        load time =    3619.52 ms
llama_print_timings:      sample time =       0.79 ms /     7 runs   (    0.11 ms per token,  8827.24 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =     271.79 ms /     7 runs   (   38.83 ms per token,    25.76 tokens per second)
llama_print_timings:       total time =     285.78 ms /     8 tokens


inference time: 288.7ms

 vertical cylindrical @handle


In [183]:
def processLLMOutput(llm_output: str) -> str:
    cleanText = llm_output
    newLineIdx = cleanText.find('\n')
    if newLineIdx != -1:
        cleanText = cleanText[:newLineIdx]

    cleanText = re.sub('[^A-Za-z0-9\"\':\{\}\[\]\_\-\,\.\(\)\;\s]+', '', cleanText)
    terminator = cleanText.find(';')
    if terminator != -1:
        cleanText = cleanText[:terminator]
    cleanText = cleanText.lower().strip(' ')
    # print(cleanText)
    return cleanText

In [ ]:
# convert text to function calls, does not check validity
actions = processLLMOutput(output['choices'][0]['text']).split(';')
actions = [a.strip() for a in actions if a != '']
print(actions)

In [151]:
REQUIRED_ARGS = {
	'pick': 1,
	'pick_place': 2,
	'ignore': 0
}

def isValidPython(code: str) -> bool:
		# adapted from https://stackoverflow.com/a/11854793/4036127 and https://stackoverflow.com/a/51271704/4036127
		try:
			m = ast.parse(code)
			m_args = [a.value for a in m.body[0].value.args]
			id_name = m.body[0].value.func.value.id
			def_name = m.body[0].value.func.attr

			# hardcoded and assumed class object is in variable named: action
			if (id_name != 'action' or len(m_args) != REQUIRED_ARGS[def_name]):
				print('func def err')
				return False
		except Exception as e:
			print('pythonic check err: ', e)
			return False
		return True

```
prompt = '''
    Your job is to extract action and objects from the given task. Respond only in JSON format.
    \nDo not hello, comment, explain, or elaborate your thought processes. Your response should have two keys: actions and objects. Split the task in as many as sub-actions possible. Only mention the name of object.
    \n
    \nExample 1:
    \nTask: Pick up banana and place on the plate.
    \nYou: {"actions": ["pick", "place"], objects: ["banana", "plate"]}
    \n
    \nExample 2:
    \nTask: Pick up the blue box.
    \nYou: {"actions": ["pick"], objects: ["blue box"]}
    \n
    \n Question: Pick up the flower pot and place it inside the box.
    '''

prompt = '''
    Your job is to extract actions and objects from the given task and invoke the allowed functions sequentially. Answer only with action.ignore() if no appropriate function available or you don't know what to do.
    Do not greet, comment, explain, or elaborate your thought processes. Split the task in as many as sub-actions possible. Use semicolon to seperate the function calls.
    Always recognise task language and translate the task and the object name to English.
    Allowed functions are: action.pick(), action.pick_place(), action.ignore()
    \nExample 1:
    Task: Pick grapes and keep it on the plate.
    <|assistant|>Answer: action.pick_place("grapes", "plate")
    \nExample 2:
    Task: Pickup the grapes.
    <|assistant|>Answer: action.pick("grapes")
    \nNow translate to English and answer for task: ''' + taskQuestion

prompt = '''
    Your job is to extract actions and objects from the given task and invoke the allowed functions sequentially. Answer only with action.ignore() if no appropriate function available or you don't know what to do.
    Do not greet, comment, explain, or elaborate your thought processes. Split the task in as many as sub-actions possible. Use semicolon to seperate the function calls.
    Always recognise task language and translate the task and the object name to English.
    Allowed functions are: action.pick(object_name), action.pick_place(object_name, target_location), action.ignore()
    \nExample 1:
    Task: Pick grapes and keep it on the plate.
    <|assistant|>Answer: action.pick_place("grapes", "plate")
    \nExample 2:
    Task: Pickup the grapes.
    <|assistant|>Answer: action.pick("grapes")
    \nNow translate to English and answer for task: ''' + taskQuestion

prompt = '''
    Your job is to extract actions and objects from the given task and invoke the allowed functions sequentially. Answer only with action.ignore() if no appropriate function available or you don't know what to do.
    Do not greet, comment, explain, or elaborate your thought processes. Split the task in as many as sub-actions possible. Use semicolon to seperate the function calls.
    Always recognise task language and translate the task and the object name to English. Always check if the function is allowed.
    Allowed functions are: action.pick(object_name), action.pick_place(object_name, target_location), action.ignore()
    \nExample 1:
    Task: Pick grapes and keep it on the plate.
    <|assistant|>Answer: action.pick_place("grapes", "plate")
    \nExample 2:
    Task: Pickup the grapes.
    <|assistant|>Answer: action.pick("grapes")
    \nNow translate to English and answer for task: ''' + taskQuestion
```

## Multi-lingual test

In [178]:
QUESTIONS = [
    {
        'qset': {
            'en': 'pick the ball and place on the table'
        },
        'ans': 'action.pick_place("ball", "table")'
    },
    {
        'qset': {
            'en': 'pick the red ball and place in the blue box'
        },
        'ans': 'action.pick_place("red ball", "blue box")'
    },
    {
        'qset': {
            'en': 'pickup the tennis ball and place on the table'
        },
        'ans': 'action.pick_place("tennis ball", "table")'
    },
    {
        'qset': {
            'en': 'pickup the banana and dance'
        },
        'ans': 'action.pick("banana")'
    },
    {
        'qset': {
            'en': 'put the ball in the box'
        },
        'ans': 'action.pick_place("ball", "box")'
    },
    {
        'qset': {
            'en': 'put the bread in tiffin'
        },
        'ans': 'action.pick_place("bread", "tiffin")'
    },
    {
        'qset': {
            'en': 'put the flower pot on the table'
        },
        'ans': 'action.pick_place("flower pot", "table")'
    },
    {
        'qset': {
            'en': 'keep the apple on the plate'
        },
        'ans': 'action.pick_place("apple", "plate")'
    },
    {
        'qset': {
            'en': 'place the bread in the tiffin'
        },
        'ans': 'action.pick_place("bread", "tiffin")'
    },
    {
        'qset': {
            'en': 'throw the coke in the bin'
        },
        'ans': 'action.pick_place("coke", "bin")'
    },
    {
        'qset': {
            'en': 'grab the soldering iron'
        },
        'ans': 'action.pick("soldering iron")'
    },
    {
        'qset': {
            'en': 'grab the bottle'
        },
        'ans': 'action.pick("bottle")'
    },
    {
        'qset': {
            'en': 'Pick up the fallen pencil'
        },
        'ans': 'action.pick("pencil")'
    },
    {
        'qset': {
            'en': 'Grab the remote and toss it on the couch'
        },
        'ans': 'action.pick_place("remote", "couch")'
    },
    {
        'qset': {
            'en': 'Lift the book and lay it on the bedside table'
        },
        'ans': 'action.pick_place("book", "bedside table")'
    },
    {
        'qset': {
            'en': 'Reach for the sunglasses sitting on the table.'
        },
        'ans': 'action.pick_place("sunglasses", "table")'
    },
    {
        'qset': {
            'en': 'Grasp the towel and hang it on the hook'
        },
        'ans': 'action.pick_place("towel", "hook")'
    },
    {
        'qset': {
            'en': 'Reach for the jacket and drape it over the chair.'
        },
        'ans': 'action.pick_place("jacket", "chair")'
    },
    {
        'qset': {
            'en': 'Would you mind fetching that book from the shelf and placing it on the coffee table, please?'
        },
        'ans': 'action.pick_place("book", "coffee table")',
    },
    {
        'qset': {
            'en': 'Please put the dish in the dishwasher'
        },
        'ans': 'action.pick_place("dish", "dishwasher")'
    },
    {
        'qset': {
            'en': 'Bin the banana peel'
        },
        'ans': 'action.pick_place("banana peel", "bin")'
    },
]

In [220]:
RESULTS = []

for qblock in QUESTIONS:
    inference_times = []
    block_result = {
        'task': qblock['qset']['en'],
        'ans': qblock['ans']
    }
    for (lang, q) in qblock['qset'].items():
        if q == '':
            continue
        
        print('--> testing %s: %s' % (lang, q))
        start_time = time.perf_counter()
        output = llm(
            prompt=getPrompt(q),
            max_tokens=160,  # Generate up to 256 tokens
            stop=["<|end|>"],
            echo=False,
            temperature=0.4
        )
        end_time = time.perf_counter()
        inference_times.append( (end_time-start_time)*1000 )
        # print("inference time: %.1fms\n" % ((end_time-start_time)*1000,))
        
        llmAns = processLLMOutput(output['choices'][0]['text'])
        block_result[lang+'_raw'] = llmAns
        block_result[lang] = llmAns == qblock['ans'] and isValidPython(llmAns)

    block_result['avg_inference_ms'] = round(sum(inference_times)/len(inference_times), 1)
    RESULTS.append(block_result)
    print('average inference: %.1fms' % block_result['avg_inference_ms'])

    print('------------')

--> testing en: pick the ball and place on the table


Llama.generate: prefix-match hit

llama_print_timings:        load time =    4007.51 ms
llama_print_timings:      sample time =       1.62 ms /    12 runs   (    0.14 ms per token,  7393.72 tokens per second)
llama_print_timings: prompt eval time =     868.05 ms /    11 tokens (   78.91 ms per token,    12.67 tokens per second)
llama_print_timings:        eval time =     271.36 ms /    11 runs   (   24.67 ms per token,    40.54 tokens per second)
llama_print_timings:       total time =    1165.57 ms /    22 tokens
Llama.generate: prefix-match hit


--> testing hi: गेंद उठाओ और मेज़ पर रखो



llama_print_timings:        load time =    4007.51 ms
llama_print_timings:      sample time =       2.77 ms /    22 runs   (    0.13 ms per token,  7953.72 tokens per second)
llama_print_timings: prompt eval time =     190.23 ms /    36 tokens (    5.28 ms per token,   189.24 tokens per second)
llama_print_timings:        eval time =     527.33 ms /    21 runs   (   25.11 ms per token,    39.82 tokens per second)
llama_print_timings:       total time =     760.25 ms /    57 tokens
Llama.generate: prefix-match hit


--> testing fr: Ramasse la balle et pose-la sur la table.


...

--> testing ru: Выбросьте банановую кожуру в мусорное ведро



llama_print_timings:        load time =    4007.51 ms
llama_print_timings:      sample time =       2.96 ms /    15 runs   (    0.20 ms per token,  5067.57 tokens per second)
llama_print_timings: prompt eval time =     114.99 ms /    20 tokens (    5.75 ms per token,   173.93 tokens per second)
llama_print_timings:        eval time =     391.60 ms /    14 runs   (   27.97 ms per token,    35.75 tokens per second)
llama_print_timings:       total time =     550.28 ms /    34 tokens
Llama.generate: prefix-match hit


--> testing cn: 把香蕉皮扔进垃圾桶



llama_print_timings:        load time =    4007.51 ms
llama_print_timings:      sample time =       4.77 ms /    46 runs   (    0.10 ms per token,  9635.53 tokens per second)
llama_print_timings: prompt eval time =     119.71 ms /    27 tokens (    4.43 ms per token,   225.54 tokens per second)
llama_print_timings:        eval time =    1165.93 ms /    45 runs   (   25.91 ms per token,    38.60 tokens per second)
llama_print_timings:       total time =    1387.16 ms /    72 tokens


average inference: 661.4ms
------------


## Result

In [223]:
pd.set_option('display.max_columns', None)
RESULTS_DF = pd.DataFrame(RESULTS)
RESULTS_DF.index += 1 # to count from 1
RESULTS_DF

,task,ans,en_raw,en,hi_raw,hi,fr_raw,fr,es_raw,es,it_raw,it,hi-en_raw,hi-en,ms_raw,ms,ru_raw,ru,cn_raw,cn,avg_inference_ms
1,pick the ball and place on the table,"action.pick_place(""ball"", ""table"")","action.pick_place(""ball"", ""table"")",True,"action.pick(""grapes"")",False,"action.pick_place(""ball"", ""table"")",True,"action.pick_place(""ball"", ""table"")",True,"action.pick_place(""ball"", ""table"")",True,"action.pick_place(""grapes"", ""table"")",False,"action.pick_place(""ball"", ""table"")",True,"action.pick_place("""", """")",False,"action.pick_place(""ball"", ""table"")",True,551.2
2,pick the red ball and place in the blue box,"action.pick_place(""red ball"", ""blue box"")","action.pick_place(""red ball"", ""blue box"")",True,"action.pick(""red grapes"") action.pick_place(""r...",False,"action.pick(""red ball"")",False,"action.pick_place(""red ball"", ""blue box"")",True,"action.pick_place(""red ball"", ""blue box"")",True,"action.pick_place(""ball"", ""box"")",False,"action.pick_place(""red ball"", ""blue box"")",True,"action.pick_place(""red ball"", ""blue box"")",True,"action.pick_place(""red ball"", ""blue box"")",True,531.6
3,pickup the tennis ball and place on the table,"action.pick_place(""tennis ball"", ""table"")","action.pick_place(""tennis ball"", ""table"")",True,"action.pick_place(""grapes"", ""table"")",False,"action.pick_place(""tennis ball"", ""table"")",True,"action.pick_place(""tennis ball"", ""table"")",True,"action.pick_place(""tennis ball"", ""table"")",True,"action.pick(""tennis ball"")",False,"action.pick_place(""tenis ball"", ""table"")",False,"action.pick_place(""tennis ball"", ""table"")",True,"action.pick_place(""web tennis ball"", ""table"")",False,514.0
4,pickup the banana and dance,"action.pick(""banana"")","action.pick(""banana"")",True,"action.pick_place(""kala"", ""above"")",False,"action.pick(""banana"")",True,"action.pick(""banana"")",True,"action.pick(""banana"")",True,"action.pick(""grapes"")",False,"action.pick_place(""bananas"", ""location where t...",False,"action.pick(""banana"")",True,ignore,False,751.7
5,put the ball in the box,"action.pick_place(""ball"", ""box"")","action.pick_place(""ball"", ""box"")",True,"action.pick_place(""grapes"", ""box"")",False,"action.pick_place(""ball"", ""box"")",True,"action.pick_place(""ball"", ""box"")",True,"action.pick_place(""ball"", ""box"")",True,"action.pick_place(""ball"", ""box"")",True,"action.pick_place(""ball"", ""box"")",True,"action.pick_place("""", """")",False,"action.pick_place(""ball"", ""box"")",True,424.8
6,put the bread in tiffin,"action.pick_place(""bread"", ""tiffin"")","action.pick_place(""bread"", ""tiffin"")",True,"action.pick(""brad"")",False,"action.pick_place(""bread"", ""tiffin"")",True,"action.pick_place(""pan"", ""dish rack"")",False,"action.pick_place(""bread"", ""lunchbox"")",False,"action.pick_place(""bread"", ""tiffin"")",True,"action.pick_place(""roti"", ""tiffin"")",False,"action.pick_place("""", ""-"")",False,"action.pick_place("""", """")",False,486.4
7,put the flower pot on the table,"action.pick_place(""flower pot"", ""table"")","action.pick_place(""flower pot"", ""table"")",True,"action.pick_place(""flower bouquet"", ""table"")",False,"action.pick_place(""pot of flowers"", ""table"")",False,"action.pick_place(""plant"", ""table"")",False,"action.pick_place(""vaso di fiori"", ""tavolo"")",False,"action.pick_place(""flower pot"", ""table"")",True,"action.pick_place(""passion fruit"", ""table"")",False,"action.pick_place(""flower pot"", ""table"")",True,"action.pick_place(""flower vase"", ""table"")",False,487.1
8,keep the apple on the plate,"action.pick_place(""apple"", ""plate"")","action.pick_place(""apple"", ""plate"")",True,"action.pick_place("""", ""plate"")",False,"action.pick_place(""apple"", ""plate"")",True,"action.pick_place(""apple"", ""plate"")",True,"action.pick_place(""apple"", ""plate"")",True,"action.pick_place(""grapes"", ""plate"")",False,"action.pick_place(""paper"", ""on top of

In [224]:
correct_count = {}
for lang in QUESTIONS[0]['qset']:
    correct_count[lang] = 0

num_of_questions = len(QUESTIONS)

print('=== RESULT ===')
for lang in correct_count:
    if not lang in RESULTS_DF.columns:
        continue

    correct_count[lang] = RESULTS_DF[lang].sum()
    print('%s: %d/%d = %.1f%%' % (lang, correct_count[lang], num_of_questions, (correct_count[lang]/num_of_questions)*100))


=== RESULT ===
en: 20/21 = 95.2%
hi: 0/21 = 0.0%
fr: 11/21 = 52.4%
es: 11/21 = 52.4%
it: 11/21 = 52.4%
hi-en: 6/21 = 28.6%
ms: 5/21 = 23.8%
ru: 9/21 = 42.9%
cn: 6/21 = 28.6%


### Another prompt

In [237]:
def getPrompt2(taskQuestion: str) -> str:
    return '''
        <|user|>\nYour job is to extract actions and objects from the given task and invoke the allowed functions sequentially. Answer only with action.ignore() if no appropriate function available or you don't know what to do.
        Do not greet, comment, explain, or elaborate your thought processes. Split the task in as many as sub-actions possible. Use semicolon to seperate the function calls.
        Always recognise task language and translate the task and the object name to English.
        Allowed functions are: action.pick(), action.pick_place(), action.ignore()
        \nExample 1:
        Task: pickup toy car and keep inside the box
        Answer: action.pick_place("toy car", "box")
        \nExample 2:
        Task: grab the packet
        Answer: action.pick("packet")
        \nNow translate and answer for task: ''' + taskQuestion + '<|end|>\n<|assistant|>'

In [240]:
RESULTS_2 = []

for qblock in QUESTIONS:
    inference_times = []
    block_result = {
        'task': qblock['qset']['en'],
        'ans': qblock['ans']
    }
    for (lang, q) in qblock['qset'].items():
        if q == '':
            continue
        
        print('--> testing %s: %s' % (lang, q))
        start_time = time.perf_counter()
        output = llm(
            prompt=getPrompt2(q),
            max_tokens=160,  # Generate up to 256 tokens
            stop=["<|end|>"],
            echo=False,
            temperature=0.4
        )
        end_time = time.perf_counter()
        inference_times.append( (end_time-start_time)*1000 )
        # print("inference time: %.1fms\n" % ((end_time-start_time)*1000,))
        
        llmAns = processLLMOutput(output['choices'][0]['text'])
        block_result[lang+'_raw'] = llmAns
        block_result[lang] = llmAns == qblock['ans'] and isValidPython(llmAns)

    block_result['avg_inference_ms'] = round(sum(inference_times)/len(inference_times), 1)
    RESULTS_2.append(block_result)
    print('average inference: %.1fms' % block_result['avg_inference_ms'])

    print('------------')

--> testing en: pick the ball and place on the table


Llama.generate: prefix-match hit

llama_print_timings:        load time =    4007.51 ms
llama_print_timings:      sample time =       1.45 ms /    12 runs   (    0.12 ms per token,  8264.46 tokens per second)
llama_print_timings: prompt eval time =     303.25 ms /    11 tokens (   27.57 ms per token,    36.27 tokens per second)
llama_print_timings:        eval time =     275.88 ms /    11 runs   (   25.08 ms per token,    39.87 tokens per second)
llama_print_timings:       total time =     602.34 ms /    22 tokens
Llama.generate: prefix-match hit


--> testing hi: गेंद उठाओ और मेज़ पर रखो



llama_print_timings:        load time =    4007.51 ms
llama_print_timings:      sample time =       2.86 ms /    21 runs   (    0.14 ms per token,  7340.09 tokens per second)
llama_print_timings: prompt eval time =     189.50 ms /    36 tokens (    5.26 ms per token,   189.97 tokens per second)
llama_print_timings:        eval time =     499.31 ms /    20 runs   (   24.97 ms per token,    40.06 tokens per second)
llama_print_timings:       total time =     732.29 ms /    56 tokens
Llama.generate: prefix-match hit


--> testing fr: Ramasse la balle et pose-la sur la table.


...

--> testing ru: Выбросьте банановую кожуру в мусорное ведро



llama_print_timings:        load time =    4007.51 ms
llama_print_timings:      sample time =       1.60 ms /    17 runs   (    0.09 ms per token, 10618.36 tokens per second)
llama_print_timings: prompt eval time =     114.94 ms /    20 tokens (    5.75 ms per token,   174.01 tokens per second)
llama_print_timings:        eval time =     397.73 ms /    16 runs   (   24.86 ms per token,    40.23 tokens per second)
llama_print_timings:       total time =     541.19 ms /    36 tokens
Llama.generate: prefix-match hit


--> testing cn: 把香蕉皮扔进垃圾桶



llama_print_timings:        load time =    4007.51 ms
llama_print_timings:      sample time =       1.51 ms /    17 runs   (    0.09 ms per token, 11250.83 tokens per second)
llama_print_timings: prompt eval time =     116.47 ms /    27 tokens (    4.31 ms per token,   231.82 tokens per second)
llama_print_timings:        eval time =     395.28 ms /    16 runs   (   24.70 ms per token,    40.48 tokens per second)
llama_print_timings:       total time =     535.65 ms /    43 tokens


average inference: 678.5ms
------------


In [242]:
pd.set_option('display.max_columns', None)
RESULTS2_DF = pd.DataFrame(RESULTS_2)
RESULTS2_DF.index += 1 # to count from 1
display(RESULTS2_DF)

correct_count = {}
for lang in QUESTIONS[0]['qset']:
    correct_count[lang] = 0

num_of_questions = len(QUESTIONS)

print('=== RESULT ===')
for lang in correct_count:
    if not lang in RESULTS2_DF.columns:
        continue

    correct_count[lang] = RESULTS2_DF[lang].sum()
    print('%s: %d/%d = %.1f%%' % (lang, correct_count[lang], num_of_questions, (correct_count[lang]/num_of_questions)*100))


,task,ans,en_raw,en,hi_raw,hi,fr_raw,fr,es_raw,es,it_raw,it,hi-en_raw,hi-en,ms_raw,ms,ru_raw,ru,cn_raw,cn,avg_inference_ms
1,pick the ball and place on the table,"action.pick_place(""ball"", ""table"")","action.pick_place(""ball"", ""table"")",True,"action.pick(""gift"")",False,"action.pick(""ball"")",False,"action.pick_place(""ball"", ""table"")",True,"action.pick(""palla"")",False,"action.pick(""uthao"")",False,"action.pick(""bola"")",False,"action.pick_place("""", """")",False,"action.pick(""ball"")",False,578.3
2,pick the red ball and place in the blue box,"action.pick_place(""red ball"", ""blue box"")","action.pick_place(""red ball"", ""blue box"")",True,"action.pick_place(""ball"", ""blue box"")",False,"action.pick(""red ball"")",False,"action.pick_place(""red ball"", ""blue box"")",True,"action.pick(""red ball"")",False,"action.pick_place(""ball"", ""box"")",False,"action.pick_place(""red ball"", ""blue box"")",True,"action.pick_place(""red ball"", ""blue box"")",True,"action.pick(""red ball"")",False,534.6
3,pickup the tennis ball and place on the table,"action.pick_place(""tennis ball"", ""table"")","action.pick_place(""tennis ball"", ""table"")",True,"action.pick("" "")",False,"action.pick_place(""tennis ball"", ""table"")",True,"action.pick_place(""tennis ball"", ""table"")",True,"action.pick(""tennis ball"")",False,"action.pick(""tennis ball"")",False,"action.pick_place(""tennis ball"", ""table"")",True,"action.pick_place(""tennis ball"", ""table"")",True,"action.pick_place(""tennis ball"", ""table"")",True,613.9
4,pickup the banana and dance,"action.pick(""banana"")","action.pick_place(""banana"")",False,"action.pick_place(""kala"", ""above"")",False,"action.pick(""banana"")",True,"action.pick(""platano"")",False,"action.pick(""banana"")",True,"action.pick(""karo"")",False,"action.pick(""pisang"")",False,"action.pick(""banana"")",True,"action.pick(""banana"")",True,537.5
5,put the ball in the box,"action.pick_place(""ball"", ""box"")","action.pick_place(""ball"", ""box"")",True,"action.pick_place(""bag"", ""box"")",False,"action.pick_place(""ball"", ""box"")",True,"action.pick_place(""pelota"", ""caja"")",False,"action.pick_place(""ball"", ""box"")",True,"action.pick(""ball"")",False,"action.pick_place(""ball"", ""container"")",False,"action.pick_place("""", """")",False,"action.pick_place(""ball"", ""box"")",True,437.0
6,put the bread in tiffin,"action.pick_place(""bread"", ""tiffin"")","action.pick_place(""bread"", ""tiffin"")",True,"action.pick_place(""breed"", ""pocket"")",False,"action.pick_place(""bread"", ""tiffin"")",True,"action.pick_place(""pan"", ""fiambrera"")",False,"action.pick_place(""pan"", ""lunch box"")",False,"action.pick(""bread"")",False,"action.pick_place(""roti"", ""tiffin"")",False,"action.pick_place(""bread"", ""lunch box"")",False,"action.pick_place("""", """")",False,504.9
7,put the flower pot on the table,"action.pick_place(""flower pot"", ""table"")","action.pick_place(""flower pot"", ""table"")",True,"action.pick_place(""flower bouquet"", ""table"")",False,"action.pick(""pot of flowers"")",False,"action.pick_place(""maceta"", ""mesa"")",False,"action.pick_place(""vaso di fiori"", ""tavolo"")",False,"action.pick_place(""flower pot"", ""table"")",True,"action.pick_place(""flower pot"", ""table"")",True,"action.pick_place(""colored pot"", ""table"")",False,"action.pick_place(""flower vase"", ""table"")",False,528.0
8,keep the apple on the plate,"action.pick_place(""apple"", ""plate"")","action.pick_place(""apple"", ""plate"")",True,"action.pick_place(""apple"", ""plate"")",True,"action.pick_place(""apple"", ""plate"")",True,"action.pick_place(""apple"", ""plate"")",True,"action.pick_place(""apple"", ""plate"")",True,"action.pick(""plate"")",False,"action.pick_place(""epal di atas pinggan"", ""sto...",False,"action.pick(""apple"")",False,"action.pick_place(""apple"", ""plate"")",True,458.7
9,place the bread in the tiffin,"action.pick_place(""bread"", ""tiffin"")","action.pick_place(""brea

=== RESULT ===
en: 15/21 = 71.4%
hi: 2/21 = 9.5%
fr: 8/21 = 38.1%
es: 5/21 = 23.8%
it: 5/21 = 23.8%
hi-en: 6/21 = 28.6%
ms: 4/21 = 19.0%
ru: 6/21 = 28.6%
cn: 5/21 = 23.8%


## Export Result

In [252]:
# RESULTS_DF.to_csv(path_or_buf='./llm_results/phi3mini-q4-result1.csv')
# RESULTS2_DF.to_csv(path_or_buf='./llm_results/phi3mini-q4-result2.csv')

PROMPTS_TEMPLATE = {
    'prompt1': [getPrompt('--TASK HERE--')],
    'prompt2': [getPrompt2('--TASK HERE--')]
}
PROMPTS_TEMPLATE_DF = pd.DataFrame(data=PROMPTS_TEMPLATE)

with pd.ExcelWriter(path='./llm_results/phi3mini-q4-results.xlsx') as writer:
    PROMPTS_TEMPLATE_DF.to_excel(writer, sheet_name='prompts-template')
    RESULTS_DF.to_excel(writer, sheet_name='prompt1-result')
    RESULTS2_DF.to_excel(writer, sheet_name='prompt2-result')


## Dynamic Translation Insertion

In [162]:
def insertTranslationToQuestions(translations: list, lang_code: str) -> None:
    if len(translations) != len(QUESTIONS):
        print('Length mismatch')
        return None
    
    i = 0
    for qblock in QUESTIONS:
        qblock['qset'][lang_code] = translations[i]
        i += 1
    
    return None

In [125]:
# print en questions to insert in translator AI.
temp = []
for qblock in QUESTIONS:
    temp.append(qblock['qset']['en'])

temp

['pick the ball and place on the table',
 'pick the red ball and place in the blue box',
 'pickup the tennis ball and place on the table',
 'pickup the banana and dance',
 'put the ball in the box',
 'put the bread in tiffin',
 'put the flower pot on the table',
 'keep the apple on the plate',
 'place the bread in the tiffin',
 'throw the coke in the bin',
 'grab the soldering iron',
 'grab the bottle',
 'Pick up the fallen pencil',
 'Grab the remote and toss it on the couch',
 'Lift the book and lay it on the bedside table',
 'Reach for the sunglasses sitting on the table.',
 'Grasp the towel and hang it on the hook',
 'Reach for the jacket and drape it over the chair.',
 'Would you mind fetching that book from the shelf and placing it on the coffee table, please?',
 'Please put the dish in the dishwasher',
 'Bin the banana peel']

In [209]:
# translated by google bard
fr_bard = [
    "Ramasse la balle et pose-la sur la table.",  # pick the ball and place on the table
    "Ramasse la balle rouge et mets-la dans la boîte bleue.",  # pick the red ball and place in the blue box
    "Ramasse la balle de tennis et pose-la sur la table.",  # pickup the tennis ball and place on the table
    "Ramasse la banane et danse.",  # pickup the banana and dance (unchanged)
    " Mets la balle dans la boîte.",  # put the ball in the box
    " Mets le pain dans le tiffin.",  # put the bread in tiffin (assuming tiffin is a lunchbox)
    "Pose le pot de fleurs sur la table.",  # put the flower pot on the table
    "Garde la pomme sur l'assiette.",  # keep the apple on the plate
    "Place le pain dans le tiffin.",  # place the bread in the tiffin
    "Jette le coca dans la poubelle.",  # throw the coke in the bin
    "Attrape le fer à souder.",  # grab the soldering iron
    "Attrape la bouteille.",  # grab the bottle
    "Ramasse le crayon tombé.",  # Pick up the fallen pencil
    "Attrape la télécommande et jette-la sur le canapé.",  # Grab the remote and toss it on the couch
    "Soulève le livre et pose-le sur la table de chevet.",  # Lift the book and lay it on the bedside table
    "Attrape les lunettes de soleil posées sur la table.",  # Reach for the sunglasses sitting on the table
    "Saisis la serviette et accroche-la au crochet.",  # Grasp the towel and hang it on the hook
    "Attrape la veste et drape-la sur la chaise.",  # Reach for the jacket and drape it over the chair
    "Pourriez-vous avoir la gentillesse de prendre ce livre sur l'étagère et de le poser sur la table basse, s'il vous plaît ?",  # Would you mind fetching that book from the shelf and placing it on the coffee table, please?
    "Veuillez mettre le plat au lave-vaisselle, s'il vous plaît.",  # Please put the dish in the dishwasher
    "Jette la peau de banane à la poubelle.",  # Bin the banana peel
]

fr_chatgpt = [
    'Prenez la balle et placez-la sur la table',
    'Prenez la balle rouge et placez-la dans la boîte bleue',
    'Prenez la balle de tennis et placez-la sur la table',
    'Prenez la banane et dansez',
    'Mettez la balle dans la boîte',
    'Mettez le pain dans la boîte à lunch',
    'Placez le pot de fleurs sur la table',
    'Gardez la pomme sur l\'assiette',
    'Placez le pain dans la boîte à lunch',
    'Jetez le Coca à la poubelle',
    'Attrapez le fer à souder',
    'Attrapez la bouteille',
    'Ramassez le crayon tombé',
    'Attrapez la télécommande et jetez-la sur le canapé',
    'Soulevez le livre et posez-le sur la table de chevet',
    'Attrapez les lunettes de soleil posées sur la table.',
    'Saisissez la serviette et accrochez-la au crochet',
    'Attrapez la veste et posez-la sur la chaise.',
    'Pourriez-vous aller chercher ce livre sur l\'étagère et le mettre sur la table basse, s\'il vous plaît?',
    'S\'il vous plaît, mettez la vaisselle dans le lave-vaisselle',
    'Jetez l\'épluchure de banane à la poubelle'
]

es_bard = [
    'Coge la pelota y colócala en la mesa.',
    'Coge la pelota roja y colócala en la caja azul.',
    'Recoge la pelota de tenis y colócala en la mesa.',
    'Recoge el plátano y baila.',
    'Pon la pelota en la caja.',
    'Pon el pan en la fiambrera.',
    'Pon la maceta en la mesa.',
    'Deja la manzana en el plato.',
    'Coloca el pan en la fiambrera.',
    'Tira la coca en la basura.',
    'Agarra el soldador.',
    'Agarra la botella.',
    'Recoge el lápiz que se ha caído.',
    'Coge el mando a distancia y tíralo en el sofá.',
    'Levanta el libro y déjalo en la mesita de noche.',
    'Coge las gafas de sol que están en la mesa.',
    'Toma la toalla y cuélgala en el gancho.',
    'Coge la chaqueta y colócala sobre la silla.',
    '¿Te importaría coger ese libro del estante y ponerlo en la mesa de centro, por favor?',
    'Por favor, pon el plato en el lavavajillas.',
    'Tira la cáscara de plátano a la basura.'
]

it_bard = [
    'Prendi la palla e mettila sul tavolo.',
    'Prendi la palla rossa e mettila nella scatola blu.',
    'Raccogli la palla da tennis e mettila sul tavolo.',
    'Raccogli la banana e balla.',
    'Metti la palla nella scatola.',
    'Metti il pane nella scatola del pranzo.',
    'Metti il vaso di fiori sul tavolo.',
    'Tieni la mela sul piatto.',
    'Metti il pane nella scatola del pranzo.',
    'Getta la coca nella spazzatura.',
    'Prendi il saldatore.',
    'Prendi la bottiglia.',
    'Raccogli la matita caduta.',
    'Prendi il telecomando e gettalo sul divano.',
    'Solleva il libro e poggialo sul comodino.',
    'Prendi gli occhiali da sole che sono sul tavolo.',
    'Afferra l\'asciugamano e appendilo al gancio.',
    'Prendi la giacca e drappeggiala sulla sedia.',
    'Ti dispiace prendere quel libro dallo scaffale e metterlo sul tavolino, per favore?',
    'Per favore, metti il piatto in lavastoviglie.',
    'Getta la buccia di banana nella spazzatura.'
]

hi_chatgpt = [
    'गेंद उठाओ और मेज़ पर रखो',
    'लाल गेंद उठाओ और नीली डिब्बे में रखो',
    'टेनिस गेंद उठाओ और मेज़ पर रखो',
    'केला उठाओ और नाचो',
    'गेंद को डिब्बे में रखो',
    'टिफिन में ब्रेड डालो',
    'फूलदान को मेज़ पर रखो',
    'सेब को प्लेट पर रखो',
    'ब्रेड को टिफिन में रखो',
    'कोक को कूड़े में डालो',
    'सोल्डरिंग आयरन पकड़ो',
    'बोतल को पकड़ो',
    'गिरी हुई पेंसिल उठाओ',
    'रिमोट पकड़ो और सोफ़े पर फेंक दो',
    'किताब उठाओ और बिस्तर के ऊपर रख दो',
    'मेज़ पर बैठे सनग्लास को छूने के लिए हाथ बढ़ाओ',
    'तौलिया पकड़ो और हुक पर लटका दो',
    'जैकेट के लिए हाथ बढ़ाओ और कुर्सी पर ढाल दो',
    'क्या आप उस किताब को शेल्फ से लेकर कॉफी टेबल पर रख सकते हैं?',
    'कृपया डिश को डिशवॉशर में डालें',
    'केले की छिलका कूड़े में डाल दो'
]

hi_en_bard = [
    'gend ko uthao aur table pe rakho',
    'lal ball ko uthao aur neele box mein rakho',
    'tennis ball pick karo and table pe rakho',
    'kela pick karo and dance karo',
    'ball box mein rakho',
    'bread ko tiffin mein rakho',
    'flower pot table pe rakho',
    'seb plate pe rakho',
    'bread ko tiffin mein rakho',
    'coke ko dustbin mein fenk do',
    'soldering iron pakdo',
    'bottle pakdo',
    'giri hui pencil uthao',
    'remote uthao and sofa pe phenko',
    'pustak uthao and bedside table pe rakho',
    'table pe rakhe hue sunglasses le lo',
    'towel pakdo and hook pe taango',
    'jacket pakdo and kursi pe latkao',
    'kya aap woh book shelf se lekar coffee table pe rakh sakte hain, please?',
    'kripya, bartan ko dishwasher mein rakhen',
    'kela ke chilke ko dustbin mein fenk do'
]

ms_bard = [
    'Ambil bola dan letakkan di atas meja.',
    'Ambil bola merah dan letakkan di dalam kotak biru.',
    'Ambil bola tenis dan letakkan di atas meja.',
    'Ambil pisang dan menari.',
    'Masukkan bola ke dalam kotak.',
    'Masukkan roti ke dalam tiffin.',
    'Letakkan pasu bunga di atas meja.',
    'Simpan epal di atas pinggan.',
    'Letakkan roti ke dalam tiffin.',
    'Buang coke ke dalam tong sampah.',
    'Pegang seterika pemateri.',
    'Pegang botol.',
    'Ambil pensel yang terjatuh.',
    'Ambil remote dan campakkannya ke atas sofa.',
    'Angkat buku dan letakkannya di atas meja tepi katil.',
    'Ambil cermin mata hitam yang ada di atas meja.',
    'Pegang tuala dan gantungkannya pada penyangkut.',
    'Ambil jaket dan sangkutkannya di atas kerusi.',
    'Bolehkah anda tolong ambil buku itu dari rak dan letakkannya di atas meja kopi, sila? (Bolehkan anda tolong ambil buku itu dari rak dan letakkannya di atas meja kopi, sila?)',
    'Silakan masukkan pinggan ke dalam mesin basuh pinggan mangkuk.',
    'Buang kulit pisang ke dalam tong sampah.'
]

ru_bard = [
    'Возьмите мяч и положите его на стол',
    'Возьмите красный мяч и положите его в синюю коробку',
    'Поднимите теннисный мяч и положите его на стол',
    'Возьмите банан и потанцуйте',
    'Положите мяч в коробку',
    'Положите хлеб в ланч-бокс',
    'Поставьте цветочный горшок на стол',
    'Оставьте яблоко на тарелке',
    'Положите хлеб в ланч-бокс',
    'Выбросьте кока-колу в мусорное ведро',
    'Возьмите паяльник',
    'Возьмите бутылку',
    'Поднимите упавший карандаш',
    'Возьмите пульт и бросьте его на диван',
    'Поднимите книгу и положите ее на тумбочку',
    'Возьмите солнцезащитные очки, которые лежат на столе',
    'Возьмите полотенце и повесьте его на крючок',
    'Возьмите куртку и накиньте ее на стул',
    'Не могли бы вы взять эту книгу с полки и положить ее на журнальный столик, пожалуйста?',
    'Пожалуйста, поставьте посуду в посудомоечную машину',
    'Выбросьте банановую кожуру в мусорное ведро'
]

cn_bard = [
    '拿起球，放到桌子上',
    '拿起红色的球，放到蓝色的盒子里',
    '捡起网球，放到桌子上',
    '拿起香蕉，然后跳舞',
    '把球放到盒子里',
    '把面包放到饭盒里',
    '把花盆放到桌子上',
    '把苹果放在盘子里',
    '把面包放到饭盒里',
    '把可乐扔进垃圾桶',
    '拿烙铁',
    '拿水瓶',
    '捡起掉在地上的铅笔',
    '拿遥控器扔到沙发上',
    '抬起书本，放到床头柜上',
    '拿放在桌子上的太阳眼镜',
    '拿毛巾把它挂在钩子上',
    '拿夹克衫把它披在椅子上',
    '请问可以麻烦您把那个架子上的书拿下来放到茶几上吗',
    '请把盘子放进洗碗机',
    '把香蕉皮扔进垃圾桶'
]


In [210]:
insertTranslationToQuestions(fr_bard, 'fr')
insertTranslationToQuestions(es_bard, 'es')
insertTranslationToQuestions(it_bard, 'it')
insertTranslationToQuestions(hi_chatgpt, 'hi')
insertTranslationToQuestions(hi_en_bard, 'hi-en')
insertTranslationToQuestions(ms_bard, 'ms')
insertTranslationToQuestions(ru_bard, 'ru')
insertTranslationToQuestions(cn_bard, 'cn')